In [ ]:
!pip install torch torchvision

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import os
import shutil

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [10]:
# デバッグ
model = Net()

input_img = torch.rand(1, 1, 28, 28)
res = model(input_img)
print(input_img.shape)
print(res.shape)
print(res)

torch.Size([1, 1, 28, 28])
torch.Size([1, 10])
tensor([[-2.2693, -2.3185, -2.2365, -2.2959, -2.3251, -2.3361, -2.3695, -2.2500,
         -2.3161, -2.3163]], grad_fn=<LogSoftmaxBackward0>)


In [11]:
## Mnist
INPUT_PATH = "/tmp/inputs/data"
OUTPUT_PATH = "/tmp/result/data"


# Mnistのデータセット
batch_size = 1
test_batch_size = 1
train_kwargs = {"batch_size": batch_size}
test_kwargs = {"batch_size": test_batch_size}

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
# データセットを準備する
dataset1 = datasets.MNIST(
    INPUT_PATH, train=True, download=True, transform=transform
)
dataset2 = datasets.MNIST(INPUT_PATH, train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [12]:
# 学習を実行する
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )

In [18]:
## 学習
epochs = 1
log_interval = 10000
train_kwargs = {"batch_size": 64}
test_kwargs = {"batch_size": 16}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == "cuda":
	cuda_kwargs = {"num_workers": 1, "pin_memory": True, "shuffle": True}
	train_kwargs.update(cuda_kwargs)
	test_kwargs.update(cuda_kwargs)
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.001)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, epochs + 1):
	train(log_interval, model, device, train_loader, optimizer, epoch)
	# test(model, device, test_loader)
	scheduler.step()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.170001
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.075408
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.205270
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.208055
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.040972
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.349903


In [14]:
torch.cuda.is_available()

True